# Proyecto EDO Equipo 15

## Integrantes

- Alejandro Camacho Pérez       C-212
- Carlos Arturo Pérez Cabrera   C-212
- Diana Laura Pérez Trujillo    C-212
- David Sánchez Iglesias        C-212

## Informe



## Ejercicios

Todos los ejercicios son utilizando el libro de Edwards

### Métodos y variables globales

In [5]:
import enum
import prettytable as pt
from collections.abc import Callable

EPSILON = 1e-8
SEPARATOR = "----------------------------------------------------------------------------"
FUNCTION = Callable[[float, float], float]
COORDINATES = list[tuple[float, float]]


class Method(enum.Enum):
    """
        Enum for the methods to solve the system of equations

        Values:
        -------
        
        EULER: Euler's method
        
        EULER_IMPROVED: Improved Euler's method
        
        Runge_Kutta: Runge-Kutta's method
        
    """
    Euler = 1
    Euler_Improved = 2
    Runge_Kutta = 3


def Great_Than(a:float, b:float)->bool:
    """
        Check if a is greater than b
        
        Parameters
        ----------
        a: float
            First value to compare
        
        b: float
            Second value to compare
        
        Returns
        -------
        bool
            True if a is greater than b
    """
    return (not Equal_To(a, b)) and (a - b > 0)


def Less_Than(a:float, b:float)->bool:
    """
        Check if a is less than b
        
        Parameters
        ----------
        a: float
            First value to compare
        
        b: float
            Second value to compare
        
        Returns
        -------
        bool
            True if a is less than b
    """
    return (not Equal_To(a, b)) and (a - b < 0)


def Equal_To(a:float, b:float, epsilon=EPSILON)-> bool:
    """
        Check if two values are equal
        
        Parameters
        ----------
        a: float
            First value to compare
        
        b: float
            Second value to compare
        
        epsilon: float
            Tolerance to consider the values equal
        
        Returns
        -------
        bool
            True if the values are equal
    """
    return abs(a - b) < epsilon

def Is_Best_Aproximation(a:float,b:float,c:float,epsilon=EPSILON) -> bool:
    """
        Chek's if the value a is a better aproximation of c than b
        
        Parameters
        ----------
        a: float
            Value to check if it's a better aproximation
        b: float
            Current best aproximation
        c: float
            Value to aproximate
        
        Returns
        -------
        bool
            True if a is a better aproximation than b        
    """
    a_error = c-a
    b_error = c-b
    
    return Equal_To(a,c, epsilon) and Less_Than(abs(a_error), abs(b_error))

def Euler_Method(function: FUNCTION, x: float, y: float, max: float, h: float, d: int) -> COORDINATES:
    """
        Calculate the values of the function using the Euler method

        Parameters
        ----------
        function: FUNCTION: (float, float) -> float
            Function to calculate the values

        x: float
            Initial value of x

        y: float
            Initial value of y

        max: float
            Maximum value of x

        h: float
            Step size

        d: int
            Number of decimals to round the values

        Returns
        -------
        COORDINATES: list[tuple[float, float]]
            List of tuples with the values of x and y
    """
    coordinates = []

    first_iteration = True
    while(Less_Than(x, max) or Equal_To(x, max)):

        # Save values
        coordinates.append((x, round(y, d)))

        # Update y value
        y = y+h*function(x, y)

        # Update x value
        x = x+h

    return coordinates


def Euler_Method_Improved(function: FUNCTION, x: float, y: float, max: float, h: float, d: int) -> COORDINATES:
    """
        Calculate the values of the function using the Euler Improved method

        Parameters
        ----------
        function: FUNCTION: (float, float) -> float
            Function to calculate the values

        x: float
            Initial value of x

        y: float
            Initial value of y

        max: float
            Maximum value of x

        h: float
            Step size

        d: int
            Number of decimals to round the values

        Returns
        -------
        COORDINATES: list[tuple[float, float]]
            List of tuples with the values of x and y
    """

    # Define first slope
    def K1(x, y):
        return function(x, y)

    # Define second slope
    def K2(x, y):
        y2 = y+h*K1(x, y)
        return function(x+h, y2)

    def K(x, y):
        return (1/2)*(K1(x, y)+K2(x, y))

    return Euler_Method(K, x, y, max, h, d)


def Runge_Kutta_Method(function: FUNCTION, x: float, y: float, max: float, h: float, d: int) -> COORDINATES:
    """
        Calculate the values of the function using the Runge-Kutta method

        Parameters
        ----------
        function: FUNCTION: (float, float) -> float
            Function to calculate the values

        x: float
            Initial value of x

        y: float
            Initial value of y

        max: float
            Maximum value of x

        h: float
            Step size

        d: int
            Number of decimals to round the values

        Returns
        -------
        COORDINATES: list[tuple[float, float]]
            List of tuples with the values of x and y
    """

    # Define first slope
    def K1(x, y):
        return function(x, y)

    # Define second slope
    def K2(x, y):
        y1 = y+h*(1/2)*K1(x, y)
        return function(x+h/2, y1)

    # Define third slope
    def K3(x, y):
        y3 = y+h*(1/2)*K2(x, y)
        return function(x+h/2, y3)

    # Define fourth slope
    def K4(x, y):
        y4 = y+h*K3(x, y)
        return function(x+h, y4)

    def K(x, y):
        return (1/6)*(K1(x, y)+2*K2(x, y)+2*K3(x, y)+K4(x, y))

    return Euler_Method(K, x, y, max, h, d)


def Calculate_Values(function: FUNCTION, method: Method, min: float, max: float, y: float, h_values: list[float], d: int) -> list[COORDINATES]:
    """
    Calculate the values of the function using the method specified

    Parameters
    ----------
    function: FUNCTION: (float, float) -> float
        Function to calculate the values

    method: Method
        Method to calculate the values

    min: float
        Minimum value of x

    max: float
        Maximum value of x

    y: float
        Initial value of y

    h_values: list
        List of h values

    d: int
        Number of decimals to round the values

    Returns
    -------
    list: list[COORDINATES] COORDINATES: list[tuple[float, float]]
        List of COORDINATES whit the values of calculate the selected method with all h values
    """

    # Calculate values
    coordinates_list = []
    for h in h_values:
        if(method == Method.Euler):
            coordinates_list.append(Euler_Method(function, min, y, max, h, d))
        elif(method == Method.Euler_Improved):
            coordinates_list.append(Euler_Method_Improved(
                function, min, y, max, h, d))
        else:
            coordinates_list.append(
                Runge_Kutta_Method(function, min, y, max, h, d))

    return coordinates_list


def Print_Table(coordinates_list: list[COORDINATES], h_values: list[float], method: Method, n: int, min: float, max: float, d: int, y: float):
    """
        This method print a table with the coordinates values for any h value

        Parameters
        ----------
        coordinates_list : list[COORDINATES] COORDINATES: list[tuple[float, float]]
            List of the resulting coordinates. Each element of the list is a tuple, where the first element is a list of x values and the second element is a list of y values. The list is ordered by the h value used to calculate the coordinates.

        h_values : list[float]
            The h values used to calculate the coordinates.

        method : Method
            The method used to calculate the coordinates.

        n : int
            The number of iterations.

        min : float
            The minimum value of x.

        max : float
            The maximum value of x.

        d : int
            The number of decimals to round the values.

        y : float
            The initial value of y.

        Returns
        -------
        None
    """

    # Select table name
    table_name = method.name == "Euler" and "Euler" or method.name == "Euler_Improved" and "Euler Mejorado" or "Runge-Kutta"

    # Selecting x values to print
    step_size = (max-min)/n
    x_values = [round(min + i*step_size, d) for i in range(n+1)]

    # Filling table
    epsilon = 0.01
    h_current = 0
    table = {"x": x_values}
    for coordinates_current in coordinates_list:

        # Initialize column with the name
        column_name = 'y(h={})'.format(h_values[h_current])
        table[column_name] = []

        x_index = 0

        # Fill column
        best_aproximation = (float('-inf'), 0)
        for x_y in coordinates_current:

            # Chek if the current value is the best aproximation to x vale to print
            if Is_Best_Aproximation(x_y[0],best_aproximation[0], x_values[x_index], epsilon):
                best_aproximation = x_y
            
            # If the current value is the best aproximation, add it to the table
            elif best_aproximation[0] != float('-inf'):
                table[column_name].append(best_aproximation[1])
                best_aproximation = (float('-inf'), 0)
                x_index += 1
            
            # If the current x_index is the last one, add the last value to the table
            if(x_index == len(x_values)-1):
                table[column_name].append(coordinates_current[-1][1])
                break
        h_current += 1

    # Print table
    pretty_table = pt.PrettyTable()
    pretty_table.title = table_name
    for key in table:
        pretty_table.add_column(key, table[key])
    print(pretty_table)


def Print_Deer_Info(data: list[COORDINATES], months, limit_poblation: int):
    """
        Print the information of the deer population

        Parameters
        ----------
        data : list[COORDINATES] COORDINATES: list[tuple[float, float]]
            List of the resulting coordinates. Each element of the list is a tuple, where the first element is a list of x values and the second element is a list of y values. The list is ordered by the h value used to calculate the coordinates.

        months : list
            The months used to calculate the coordinates.

        limit_poblation : int
            The limit of the deer population.

        Returns
        -------
        None
    """
    poblation = data[0][months-1][1]
    years = months/12
    print("Población de ciervos en {} años: {}".format(years, poblation))
    print("Porcentaje de población de ciervos en {} años con respecto a {}: {}%".format(
        years, limit_poblation, round(poblation/limit_poblation*100, 2)))

    print(SEPARATOR)


### Ejercicio 24, página 132

Para el problema se requiere una computadora con
impresora. En este problema de valor inicial utilice el método de Euler mejorado con tamaños de paso h = 0.1, 0.02,
0.004 y 0.0008 para aproximar con 5 cifras decimales el valor
de la solución en 10 puntos igualmente espaciados del intervalo dado. Imprima los resultados en forma tabular con los encabezados apropiados para facilitar la comparación del efecto
de variar el tamaño de paso h. Las primas representan derivadas con respecto a x.

 - $y'= \frac{x}{1+y²},y(-1)=1;-1 \leq x \leq 1$

#### Código

In [ ]:
# region Defining variables

# Function
def function(x, y):
    return x/(1+y**2)


# Initial values
x = -1
y = 1

# Interval
max = 1

# Decimals to round
d = 5

# Steps
h_values = [0.1, 0.02, 0.004, 0.0008]

# Numbers of values to print
n = 10

# endregion

# Aply method
coordinates_list = Calculate_Values(function, Method.Euler_Improved, x, max, y, h_values, d)


# Print
Print_Table(coordinates_list, h_values, Method.Euler_Improved, n, x, max,d,y)


#### Resultados

| x | y(h=0.1) | y(h=0.02) | y(h=0.004) | y(h=0.0008) |
|:---:|:---:|:---:|:---:|:---:|
| -1.0 |    1     |     1     |     1      |      1      |
| -0.8 | 0.90572  |  0.90569  |  0.90569   |   0.90569   |
| -0.6 | 0.82574  |  0.82569  |  0.82569   |   0.82569   |
| -0.4 | 0.76449  |  0.76443  |  0.76443   |   0.76443   |
| -0.2 | 0.72593  |  0.72586  |  0.72586   |   0.72586   |
| 0.0  | 0.71276  |  0.71268  |  0.71268   |   0.71268   |
| 0.2  | 0.72595  |  0.72586  |  0.72586   |   0.72586   |
| 0.4  | 0.76453  |  0.76443  |  0.76443   |   0.76443   |
| 0.6  | 0.82579  |  0.82569  |  0.82569   |   0.82569   |
| 0.8  | 0.90578  |  0.90569  |  0.90569   |   0.90569   |
| 1.0  | 1.00006  |    1.0    |    1.0     |     1.0     |

### Ejercicio 24, Página 142

Para el problema se requiere una computadora con
impresora. En estos problemas de valor inicial utilice el méto-
do de Runge-Kutta con tamaños de paso h = 0.2 , 01, 0.05 y
0.025 para aproximar a 6 cifras decimales los valores de la
solución en 5 puntos igualmente espaciados del intervalo
dado. Imprima los resultados en forma tabular con un enca-
bezado apropiado que facilite la comparación del efecto de
variar el tamaño de paso h. Las primas representan derivadas
con respecto a x.

 - $y'= \frac{x}{1+y²},y(-1)=1;-1 \leq x \leq 1$

#### Código

In [ ]:
#region Defining variables

# Function
def function(x, y):
    return x/(1+y**2)


# Initial values
x = -1
y = 1

# Interval
max = 1

# Decimals to round
d = 6

# Steps
h_values = [0.2, 0.1, 0.05, 0.025]

# Number of values to print
n=5

# endregion

# Aply method
coordinates_list = Calculate_Values(function, Method.Runge_Kutta, x, max, y, h_values, d)


# Print
Print_Table(coordinates_list, h_values, Method.Runge_Kutta, n, x, max,d,y)


#### Resultados

| x | y(h=0.2) | y(h=0.1) | y(h=0.05) | y(h=0.025) |
|:---:|:---:|:---:|:---:|:---:|
| -1.0 |    1     |    1     |     1     |     1      |
| -0.6 | 0.82569  | 0.825691 |  0.825691 |  0.825691  |
| -0.2 | 0.725856 | 0.725856 |  0.725857 |  0.725857  |
| 0.2  | 0.725856 | 0.725856 |  0.725857 |  0.725857  |
| 0.6  | 0.82569  | 0.825691 |  0.825691 |  0.825691  |
| 1.0  |   1.0    |   1.0    |    1.0    |    1.0     |

### Ejercicio 26, página 132

Suponga que
en un pequeño bosque la población de venados P(t) inicialmente es de 25 individuos y satisface la ecuación
logística  

- $\frac{dP}{dt} = 0.0225P − 0.0003P²$  

(con t en meses). Utilice el método de Euler con
una calculadora programable o una computadora para aproximar la solución a 10 años, primero con un tamaño de paso h = 1 y después con h = 0.5, redondeando
los valores aproximados de P a números enteros de venados. ¿Qué
porcentaje de la población límite de 75 venados se obtiene
después de 5 años? ¿Después de 10 años?


#### Código

In [ ]:
# region Defining variables

# Function
def function(x, y):
    return 0.0225*y-0.0003*(y**2)


# Initial values
x = 0
y = 25

# Interval
max=120

# Decimals
d=0

# Steps
h_values=[1,0.5]

# Numbers of values to print
n=12

# Aply method
deer_data = Calculate_Values(function, Method.Euler, x, max, y, h_values, d)


# Print
Print_Table(deer_data, h_values, Method.Euler, n, x, max,d,y)

# Print info
Print_Deer_Info(deer_data, 60, 75)
Print_Deer_Info(deer_data, 120, 75)



#### Resultados

| x | y(h=1) | y(h=0.5) |
|:---:|:---:|:---:|
|  0.0  |   25   |    25    |
|  10.0 |  29.0  |   29.0   |
|  20.0 |  33.0  |   33.0   |
|  30.0 |  37.0  |   37.0   |
|  40.0 |  41.0  |   41.0   |
|  50.0 |  45.0  |   45.0   |
|  60.0 |  49.0  |   49.0   |
|  70.0 |  53.0  |   53.0   |
|  80.0 |  56.0  |   56.0   |
|  90.0 |  59.0  |   59.0   |
| 100.0 |  62.0  |   62.0   |
| 110.0 |  64.0  |   64.0   |
| 120.0 |  66.0  |   66.0   |

- Población de ciervos en $5$ años: $49$
- Porcentaje de población de ciervos en $5$ años con respecto a $75: 65.33\%$

- Población de ciervos en $10$ años: $66$
- Porcentaje de población de ciervos en $10$ años con respecto a $75: 88.0\%$
